> This notebook uses the descriptive data csv and XGBoost to experiment with modeling using the metadata to see if there is any signal in that.

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import xgboost
from xgboost import XGBClassifier

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv("descriptive_data.csv")

In [3]:
data.head()

,Unnamed: 0,lesion_id,image_id,dx,dx_type,age,sex,localization,pixel_avg
0,0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,184.320017
1,1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,177.003825
2,2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,181.487038
3,3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,165.210795
4,4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,188.305992


In [5]:
data = pd.get_dummies(data, columns = ["localization"])

In [8]:
data["sex"] = data["sex"].map({"male": 0, "female": 1})

In [30]:
data["dx"].value_counts()

nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

> The features for this model will be age, sex, localization and pixel average. The baseline for the model is 67% as that is the distribution of the majority class.

In [11]:
X = data.drop(["Unnamed: 0", "lesion_id", "image_id", "dx", "dx_type"], axis = 1)

In [13]:
y = data["dx"].map({"nv": 4, "mel": 6, "bkl": 2, "bcc": 1, "akiec": 0, "vasc": 5, "df":3 })

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

ss = StandardScaler()
X_train  = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [23]:
xgb = XGBClassifier(random_state = 30)

In [24]:
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=30,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [25]:
xgb.score(X_train, y_train)

0.7182798562108907

In [26]:
xgb.score(X_test, y_test)

0.6964856230031949

> This model has an accuracy that is 3% above the baseline. It may be useful to combine this with cnn models in future tests.